In [ ]:
import pandas as pd
import numpy as np

# Groupby

As I pointed out in the first part of this lesson, tidy data is only useful if we have tools that work with it in a consistent and reproducable manner. One such tools is a `groupby` method of `DataFrame`, which provides a powerful interface to apply any operation based on groupping variables, and we will talk about it in detail in the current section.

It turns out that very frequently we need to do some operation based on a groupping variable. A common example is calculating mean of each group (e.g. performance of each subject, or performance on each type of stimuli, etc). This can be thought of as making 3 separate actions:
- Splitting the data based on a groupping variable(s)
- Applying a function to each group separately
- Combining the resulting values back together

Based on these 3 actions, this approach is called *Split-Apply-Combine* (SAC) [1].

[1] Wickham, Hadley. "The split-apply-combine strategy for data analysis." Journal of Statistical Software 40.1 (2011): 1-29.

<img src="http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/03.08-split-apply-combine.png"></img>
From ["Aggregation and groupping" chapter](http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.08-Aggregation-and-Grouping.ipynb) of ["Python Data Science Handbook"](http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/Index.ipynb) by Jake VanderPlas

A lot of operations on data can be thought of as SAC operations. These include calculating sums, means, standard deviations and other parameters of the groups' distributions; transfromations of data, such as normalization or detrending; plotting based on group, e.g. boxplots; and many other. (Some operations cannot be thought of as purely SAC, most prominently those in which data from the same group is used several times, e.g. rolling window means.)

A traditional way of doing these operations in include loops, where on each iteration a subset of data is selected and processed. Loops, however, are slow and usually require a lot of code, which makes them difficult to read, and are not easily extendible from 1 to several groupping variables.

`Groupby` is a method of `DataFrames` which makes any SAC operation easy to perform and read.

>**Note**: Tidy data is the most convenient form for making SAC operations, because you always have access to any combination of your groupping variables due to them being always separated in columns.

Let's see a toy example of using a `groupby` operation instead of a loop.

In [ ]:
df = pd.DataFrame({'group': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)})
df

Let's say I want to calculate a sum of `data` column, based on `group` variable and save it in a `Series`. I can do it with a loop:

In [ ]:
result = pd.Series()

groups = df['group'].unique()
for g in groups:
    data = df.loc[df['group']==g, 'data']
    result[g] = np.sum(data)

result

This code does the job, but it is quite long. If I try to shorten it, it will become very difficult to read:

In [ ]:
result = pd.Series()
for g in df['group'].unique():
    result[g] = np.sum(df.loc[df['group']==g, 'data'])

result

Now let's try to do the same thing with `groupby`:

In [ ]:
df.groupby('group')['data'].sum()

See that it is really short and concise and readable. Moreover, let's say I have a more complicated example with several groupping variables:

In [ ]:
df = pd.DataFrame({'group1': ['A', 'B', 'C']*3,
                   'group2': ['A']*4 + ['B']*1 + ['C']*4,
                   'data': range(9)})
df

Trying to calculate a sum based on these several groups requires significantly more code with loops. With `groupby` it is as easy as adding another groupping variable in the `groupby` attributes:

In [ ]:
result = df.groupby(['group1','group2'])['data'].sum()
result

>**Pro-tip**: You may notice that in the resulting `Series` index has 2 levels: `group1` and `group2`. This is referred to as *Hierarchical index* or `MultiIndex`, and is a way to stack several dimensions of data. We won't go much into the details of `MultiIndex` (if you wish to learn more, you may refer to [this section](http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.05-Hierarchical-Indexing.ipynb) of [Python Data Science Handbook](http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/Index.ipynb) and to [MultiIndex](http://pandas.pydata.org/pandas-docs/stable/advanced.html) section of `pandas` documentation. For our purposes we just need to know 2 things: how to index a `MultiIndex` and how to *unstack* dimensions to turn it into a table:

In [ ]:
# get an element with group1 = A and group2 = C
result[('A','C')]

In [ ]:
# unstack levels of multiindex (turn one of them into a column)
result.unstack()

Overall, `groupby` is an extremely useful tool for making group-based operations quickly and more readible. Let's see some concrete examples of how you can use it. We will work on the data in the food preferences task provided by Paolo Garlasco. Let's load it first and do some cleanup:

In [ ]:
df = pd.read_csv('data/Paolo.csv')
# drop old index column
df.drop('Unnamed: 0', axis='columns', inplace=True)
df['cond'].replace({1: 'high vs high', 2: 'low vs low', 
                    3: 'high vs low', 4: 'low vs high'}, inplace=True)
df['congr'].replace({0: 'same', 1: 'different'}, inplace=True)
df['session'].replace({0: 'fed', 1: 'hungry'}, inplace=True)
print(df.shape)
df.head()

The data contains 4 subjects:

In [ ]:
df['subj_num'].unique()

Let's calculate mean reaction time for each subject:

In [ ]:
df.groupby('subj_num')['rt'].mean()

Subjects also seem to have more that 1 session, so we might want to compute mean for each session separately:

In [ ]:
rt_subject_session = df.groupby(['subj_num','session'])['rt'].mean()
rt_subject_session

# <font color='DarkSeaGreen '>Exercise</font>
In the cell below calculate mean response for each food item.



As we saw above, `pandas` provides shortcuts to applying some frequent functions, such as `mean()`, `std()`, `count()`, `min()`, `max()`. However, we can apply any function to the groups. to do that, there are 3 methods: `aggregate()`, `transform()` and `apply()`. Each of these methods require a function (the one you want to apply to the data) as an argument.

## Aggregate
`aggregate()` can apply any function, which returns a single value for each group (in other words, it *aggregates* a group to a single value). This is what mean, std, count, min, max, and others are. Instead of writing `df.groupby('subj_num')['rt'].mean()` we could've passed a `np.mean` function to calculate means:

In [ ]:
df.groupby('subj_num')['rt'].aggregate(np.mean)

You can also specify several functions in a list, and `aggregate()` will return results of all of them in a neat table:

In [ ]:
df.groupby('subj_num')['rt'].aggregate([np.mean, np.std, np.median])

More importantly, you can create any function and pass it to `aggregate()` and the function will be applied to each group. The only limitation is that code will assume that the function returns a single value, e.g. calculate half of mean:

In [ ]:
def hafl_mean(x):
    """Calculate half of the mean"""
    mean = np.mean(x)
    return mean/2

df.groupby('subj_num')['rt'].aggregate(hafl_mean)

## Transform
`transform()` works exactly like `aggregate()`, but it expects a function to return a `Series` or an `array` of the same size as input. It will handle the cases when you want to tranform the data. For example, we could subtract the mean reaction time for each subject:

In [ ]:
def subtract_mean(x):
    return x - np.mean(x)

df['rt_minus_mean'] = df.groupby('subj_num')['rt'].transform(subtract_mean)
df.head()

# <font color='DarkSeaGreen '>Exercise</font>
In the cell below calculate standard score (*z-score*) on reaction time for each subject using `groupby` and `transform`. Save z scores to a new column. 

See which 10 items require highest reaction times on average in all subjects.

All other cases, which don't fall within `aggregate` and `transform` can be handled by `apply` method. In reality, `apply` can act as both `aggregate` and `transform` in most circumstances, but it is slower (because it cannot assume output shape) and cannot do certain things, for example, aggregate several functions at once like `aggregate` method can.

## Looping with groupby
`groupby`-`apply` combination lets us in general avoid loops, but sometimes you might still need to use them. For example, this can happen when you want to do plotting by group. `groupby` can also simplify that, because it supports iteration through itself. When you do it, on each iteration it will give 2 values: one for the name of the group (basically, groupping variable value) and the values of the group.

In [ ]:
# assign a grouby object to a variable
groupped = df.groupby('subj_num')['rt']
# iterate through groupby object
for name, data in groupped:
    # groupping variable value
    print(name)
    # shape of the data: in this case the 'rt' values for each group
    print(data.shape)

# `DataFrame` and `Series` transformations
Now that you know the power of `groupby` and having data in a tidy format, let's talk about how to get there. In general, you should become comfortable with transforming your data to any shape you want, because the tools you might want to use, won't necessarily work with tidy data. `pandas` provides a lot of ways to tranform `Series` and `DataFrame` objects.

## `Set`, `reset` index
Index is very useful for retrieving values, but also for other things. For example, as we will see in the visualization lesson, when plotting a `Series`, index will be automatically assumed to be the X axis, and the values will become the Y axis. This is useful for quick exploratory visualization.

Main methods to interact with the index are `set_index()` and `reset_index()`. First takes a column and makes it into a new index:

In [ ]:
df_items = df.set_index('item')
df_items.head()

In [ ]:
# if you say append=True, you can keep the old index too, which will result in a MultiIndex
df.set_index('item', append=True).head()

`reset_index()` will make the old index into a columns and instead create a new index, which has values from `0` to the number of rows minus 1:

In [ ]:
# our DataFrame indexed by items
df_items.head()

In [ ]:
# let's reset index
df_items.reset_index().head()

These two methods make working with index very dynamic -- you can set it and reset it to become a normal column again whenever you need. You can also set several columns (pass them as a list to `set_index`) and create a `MultiIndex`.

## Melt
The concept of melting is related to tidying the data. `melt` function takes all columns of the `DataFrame` and creates 2 columns from them: one with groupping variable (former name of the column) and another with the value variable. If applied correctly, the resulting *molten* `DataFrame` will be tidy.

Let's see a toy example:

In [ ]:
untidy = pd.DataFrame({'treatment_a':[np.nan, 16, 3],'treatment_b':[2,11,1]})
untidy

In [ ]:
# let's melt
pd.melt(untidy)

Note how the data is reshaped. What were the names of the columns in the untidy `DataFrame` (`treatment_a` and `treatment_b`) are now the groupping variable. The values inside the table are now all in the single "value" column.

In [ ]:
# you can also specify the names of the resulting columns
pd.melt(untidy, var_name='treatment', value_name='measurement')

Frequently you want to melt only certain columns, because some are already groupping variable. Specify them as `id_vars` in the `melt` function and they will not be changed:

In [ ]:
# in this example "person" is already a separated variable
untidy = pd.DataFrame({'treatment_a':[np.nan, 16, 3],'treatment_b':[2,11,1], 
                      'person':['John Smith', 'Jane Doe','Mary Johnson']})
untidy

In [ ]:
pd.melt(untidy, id_vars='person', var_name='treatment', value_name='measurement')

Let's see another example, taken directly from the [lesson on tidy data](http://nbviewer.jupyter.org/github/antopolskiy/sciprog/blob/master/002_data_organization_00_slides.ipynb):

In [ ]:
income_untidy = pd.read_csv('data\pew.csv')
print(income_untidy.shape)
income_untidy.head()

In this case all columns except for `religion` have the same variable (count of people who belongs to this group), so we keep `religion` and melp all other columns:

In [ ]:
income_tidy = pd.melt(income_untidy,id_vars='religion',var_name='income',value_name='count')
print(income_tidy.shape)
income_tidy.head()

# Pivot table
Pivoting is another way of transforming the `DataFrames`, which is usually used to tranform a tidy `DataFrame` in some other form. For example, it can be used to undo melting. Using method `pivot_table` is easy: simply think about which column you want to have as and index and which as columns.

In [ ]:
# molten dataframe
income_tidy.head()

In [ ]:
# pivoting to undo melting
income_tidy.pivot_table(columns='income', index='religion')

But pivoting can achieve much more than that. Let's look at another example. This dataset contains number of births for each day from 1969 to 2008:

In [ ]:
births = pd.read_csv('data/births.csv')
births.head()

Let's say we want to calculate the total number of births for each year for boys and girls to see how the gender proportions change over the years. We could achieve it with `groupby`:

In [ ]:
births.groupby(['year','gender'])['births'].sum().head(10)

We could then use `unstack` on the resulting `Series` to create a nice table:

In [ ]:
births.groupby(['year','gender'])['births'].sum().unstack()

Pivot table can do the same and in some cases can be more readable, because when we pivot we don't need to think about groupping, but instead we think about what kind of table we want to get in the end. In this case I think to myself: "I want *year* to be the index, *genders* will be the columns. I will take the *births* columns and *sum* them up for each resulting group". The syntax of `pivot_table` repeats this thinking almost exactly:

In [ ]:
births_year_gender = births.pivot_table(index='year', columns='gender', values='births', aggfunc=np.sum)
births_year_gender

Let's see another example on Paolo's food preference data.

In [ ]:
df.head()

I want to create a table with mean reaction times with rows being session type and columns being the condition.

In [ ]:
df.pivot_table(values='rt', index='session', columns='cond', aggfunc=np.mean)

# <font color='DarkSeaGreen '>Exercise</font>
Using `births` dataset, create a table in which there would be total number of births for each month for each year. Do it using `groupby-aggregate-unstack` and using `pivot_table`.

# <font color='DarkSeaGreen '>Exercise</font>
Using food preference dataset, create a table in which the index would be items, columns would be session type and the values would be mean response.